In [1]:
import pandas as pd
%pylab inline
import pandas_datareader.data as web
import datetime
from datetime import timedelta
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from sqlalchemy import create_engine


%load_ext sql
engine = create_engine('postgresql://localhost:5432/kylesdb')

plt.rcParams['figure.figsize'] = (14, 8)

Tesla = pd.read_sql("select * from tesla", engine)
Tesla.rename(index=str, columns={'Date':"TEdate", 'Open': 'TEopen', 'High':'TEhigh', 'Low':'TElow','Close':'TEclose', 'Volume':'TEvol'}, inplace=True)
GM = pd.read_sql("select * from \"GM\"", engine)
GM.rename(index=str, columns={'Date':"GMdate", 'Open': 'GMopen', 'High':'GMhigh', 'Low':'GMlow','Close':'GMclose', 'Volume':'GMvol'}, inplace=True)
Ford = pd.read_sql("select * from \"ford\"", engine)
Ford.rename(index=str, columns={'Date':"Fdate", 'Open': 'Fopen', 'High':'Fhigh', 'Low':'Flow','Close':'Fclose', 'Volume':'Fvol'}, inplace=True)
Daimler = pd.read_sql("select * from \"daimler\"", engine)
Daimler.rename(index=str, columns={'Date':"Ddate", 'Open': 'Dopen', 'High':'Dhigh', 'Low':'Dlow','Close':'Dclose', 'Volume':'Dvol'}, inplace=True)
Toyota = pd.read_sql("select * from \"toyota\"", engine)
Toyota.rename(index=str, columns={'Date':"TOdate", 'Open': 'TOopen', 'High':'TOhigh', 'Low':'TOlow','Close':'TOclose', 'Volume':'TOvol'}, inplace=True)
Solar = pd.read_sql("select * from \"solar\"", engine)
Solar.rename(index=str, columns={'Date':"Sdate", 'Open': 'Sopen', 'High':'Shigh', 'Low':'Slow','Close':'Sclose', 'Volume':'Svol'}, inplace=True)
SP_500 = pd.read_sql("select * from \"sp_500\"", engine)
SP_500.rename(index=str, columns={'Date':"SPdate", 'Open': 'SPopen', 'High':'SPhigh', 'Low':'SPlow','Close':'SPclose', 'Volume':'SPvol', 'Adj Close': 'SPadj_clo' }, inplace=True)
Nasdaq = pd.read_sql("select * from \"nasdaq\"", engine)
Nasdaq.rename(index=str, columns={'Date':"NQdate", 'Open': 'NQopen', 'High':'NQhigh', 'Low':'NQlow','Close':'NQclose', 'Volume':'NQvol', 'Adj Close': 'NQadj_clo' }, inplace=True)
Nasdaq.drop('NQadj_clo', axis=1, inplace=True)
SP_500.drop('SPadj_clo', axis=1, inplace=True)

Populating the interactive namespace from numpy and matplotlib


/Users/kyletoomey/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/kyletoomey/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
a=pd.merge(SP_500, Nasdaq, right_index=True, left_index=True)
b=pd.merge(Solar,a,right_index=True, left_index=True)
c=pd.merge(Toyota,b,right_index=True, left_index=True)
d=pd.merge(Daimler,c,right_index=True, left_index=True)
e=pd.merge(Ford,d,right_index=True, left_index=True)
f=pd.merge(GM,e,right_index=True, left_index=True)
master=pd.merge(Tesla,f,right_index=True, left_index=True)

In [4]:
master.head()

,TEdate,TEopen,TEhigh,TElow,TEclose,TEvol,GMdate,GMopen,GMhigh,GMlow,...,SPhigh,SPlow,SPclose,SPvol,NQdate,NQopen,NQhigh,NQlow,NQclose,NQvol
0,2015-01-02 00:00:00-05:00,222.87,223.25,213.26,219.31,4764443,2015-01-02 00:00:00-05:00,35.27,35.31,34.41,...,2072.360107,2046.040039,2058.199951,2708700000,2015-01-02 00:00:00-05:00,4760.240234,4777.009766,4698.109863,4726.810059,1435150000
1,2015-01-05 00:00:00-05:00,214.55,216.50,207.16,210.09,5368477,2015-01-05 00:00:00-05:00,34.98,35.25,34.07,...,2054.439941,2017.339966,2020.579956,3799120000,2015-01-05 00:00:00-05:00,4700.339844,4702.770020,4641.459961,4652.569824,1794470000
2,2015-01-06 00:00:00-05:00,210.06,214.20,204.21,211.28,6261936,2015-01-06 00:00:00-05:00,34.41,35.22,34.10,...,2030.250000,1992.439941,2002.609985,4460110000,2015-01-06 00:00:00-05:00,4666.850098,4667.330078,4567.589844,4592.740234,2167320000
3,2015-01-07 00:00:00-05:00,213.35,214.78,209.78,210.95,2968390,2015-01-07 00:00:00-05:00,35.23,35.95,35.04,...,2029.609985,2005.550049,2025.900024,3805480000,2015-01-07 00:00:00-05:00,4626.839844,4652.720215,4613.899902,4650.470215,1957950000
4,2015-01-08 00:00:00-05:00,212.81,213.80,210.01,210.62,3442509,2015-01-08 00:00:00-05:00,36.06,36.45,35.80,...,2064.080078,2030.609985,2062.139893,3934010000,2015-01-08 00:00:00-05:00,4689.540039,4741.379883,4688.020020,4736.189941,2105450000


In [36]:
pd.set_option('display.max_columns', 50)
master.corr()[master.corr()>.5]

SyntaxError: invalid syntax (<ipython-input-36-82a664fb6236>, line 2)